# 🚖 NCR Ride Bookings — Data Cleaning Script

## 📌 Overview
This notebook cleans and prepares the **NCR ride bookings dataset** for Exploratory Data Analysis (EDA).
The raw dataset contains booking details, ride outcomes, cancellations, incompletions, and metrics such as ride distance, booking value, and ratings.

## 🛠️ Cleaning Steps
1. **Datetime Handling**
   - Merged `Date` and `Time` into a single `Datetime` column.

2. **Identifier Cleanup**
   - Removed extra quotes from `Booking ID` and `Customer ID`.

3. **Missing Value Handling**
   - **Ride metrics (`Avg VTAT`, `Avg CTAT`, `Booking Value`, `Ride Distance`)**
     → Set to `0` if the ride was not completed.
   - **Payment Method**
     → Set to `"Not Applicable"` if the ride was not completed.
   - **Cancellation / Incompletion columns**
     → Filled with `0` (counts) or `"Not Applicable"` (reasons) when not relevant.
   - **Ratings (`Driver Ratings`, `Customer Rating`)**
     → Missing values filled with `"Not Rated"`.

4. **Categorical Cleanup**
   - Standardized text (removed spaces, consistent casing).

5. **Data Types**
   - Ensured numeric columns are proper `float`/`int` types.
   - Kept categorical columns as `string`.

6. **Export**
   - Final cleaned dataset is saved as:
     ```
     ncr_ride_bookings_cleaned.csv
     ```
   - Saved with `index=False` to avoid including the DataFrame index column in the file.

## ✅ Result
The output dataset is:
- Consistent
- Missing values logically handled
- Ready for Exploratory Data Analysis (EDA) and feature engineering.



In [46]:
import pandas as pd

In [47]:
df_raw=pd.read_csv('data/ncr_ride_bookings.csv')


In [48]:
df= df_raw.copy()

In [49]:
print(df.shape)            # (rows, columns)
df.sample(5, random_state=42)   # random spot-check


(150000, 21)


,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
59770,2024-10-25,16:24:48,"""CNR5785058""",Cancelled by Customer,"""CID7525538""",Go Sedan,Ashok Park Main,Rohini East,18.7,NaN,...,Change of plans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21362,2024-10-03,06:15:38,"""CNR5259986""",Completed,"""CID9828317""",Go Sedan,Samaypur Badli,Hero Honda Chowk,4.2,30.1,...,NaN,NaN,NaN,NaN,NaN,269.0,47.17,4.4,3.9,Cash
127324,2024-12-07,15:35:47,"""CNR1351335""",Completed,"""CID9426219""",Go Mini,Gurgaon Sector 29,Lajpat Nagar,11.3,35.1,...,NaN,NaN,NaN,NaN,NaN,352.0,9.57,4.3,4.3,Cash
140509,2024-12-06,21:05:39,"""CNR2033265""",Completed,"""CID6253371""",Go Mini,Sonipat,Rohini,8.7,42.9,...,NaN,NaN,NaN,NaN,NaN,313.0,48.18,4.3,5.0,UPI
144297,2024-12-03,09:55:48,"""CNR4076785""",Completed,"""CID4419792""",Premier Sedan,Shastri Park,Sarai Kale Khan,3.5,19.1,...,NaN,NaN,NaN,NaN,NaN,365.0,12.06,3.9,3.8,Uber Wallet


In [50]:
df['Datetime'] = pd.to_datetime(df['Date'] + " " + df['Time'])
df.drop(['Date', 'Time'], axis=1, inplace=True)


In [51]:
df['Booking ID'] = df['Booking ID'].str.replace('"', '', regex=False)
df['Customer ID'] = df['Customer ID'].str.replace('"', '', regex=False)


In [52]:
df['Cancelled Rides by Customer'] = df['Cancelled Rides by Customer'].fillna(0).astype(int)
df['Reason for cancelling by Customer'] = df['Reason for cancelling by Customer'].fillna("Not Applicable")

In [53]:
df['Cancelled Rides by Driver'] = df['Cancelled Rides by Driver'].fillna(0).astype(int)
df['Driver Cancellation Reason'] = df['Driver Cancellation Reason'].fillna("Not Applicable")

In [54]:
df['Incomplete Rides'] = df['Incomplete Rides'].fillna(0).astype(int)
df['Incomplete Rides Reason'] = df['Incomplete Rides Reason'].fillna("Not Applicable")

In [55]:
mask_completed = df['Booking Status'] == "Completed"

df.loc[~mask_completed, ['Avg VTAT', 'Avg CTAT', 'Booking Value',
                         'Ride Distance']] = df.loc[~mask_completed,
                         ['Avg VTAT', 'Avg CTAT', 'Booking Value', 'Ride Distance']].fillna(0)

df.loc[~mask_completed, 'Payment Method'] = df.loc[~mask_completed, 'Payment Method'].fillna("Not Applicable")

In [56]:
df['Driver Ratings'] = df['Driver Ratings'].fillna("Not Rated")
df['Customer Rating'] = df['Customer Rating'].fillna("Not Rated")


In [57]:
cat_cols = ['Booking Status', 'Vehicle Type', 'Pickup Location',
            'Drop Location', 'Reason for cancelling by Customer',
            'Driver Cancellation Reason', 'Incomplete Rides Reason',
            'Payment Method']

for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.title()

In [58]:
num_cols = ['Avg VTAT', 'Avg CTAT', 'Booking Value', 'Ride Distance']
df[num_cols] = df[num_cols].astype(float)

In [59]:
df.isnull().sum()

Booking ID                           0
Booking Status                       0
Customer ID                          0
Vehicle Type                         0
Pickup Location                      0
Drop Location                        0
Avg VTAT                             0
Avg CTAT                             0
Cancelled Rides by Customer          0
Reason for cancelling by Customer    0
Cancelled Rides by Driver            0
Driver Cancellation Reason           0
Incomplete Rides                     0
Incomplete Rides Reason              0
Booking Value                        0
Ride Distance                        0
Driver Ratings                       0
Customer Rating                      0
Payment Method                       0
Datetime                             0
dtype: int64

In [60]:
df.to_csv("ncr_ride_bookings_cleaned.csv", index=False)
print("✅ Cleaned dataset saved as 'ncr_ride_bookings_cleaned.csv'")


✅ Cleaned dataset saved as 'ncr_ride_bookings_cleaned.csv'
